<a href="https://colab.research.google.com/github/guifav/curso-ia-aplicada/blob/main/Analisador_de_Licita%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Extrator de Informações de Editais de Licitação
# Este código extrai informações de editais de licitação de arquivos PDF
# e as salva em um arquivo CSV usando a API GPT-4o da OpenAI

# Instalação das bibliotecas necessárias
!pip install PyPDF2 openai pandas tqdm

import os
import re
import json
import pandas as pd
import PyPDF2
import io
import base64
from openai import OpenAI
from google.colab import drive, userdata, files
from tqdm.notebook import tqdm
import time
from datetime import datetime

# Montar o Google Drive (opcional, mas útil para processar múltiplos arquivos)
drive.mount('/content/drive')

# Configurar a API da OpenAI
api_key = userdata.get('OPENAI_API_KEY')
cliente = OpenAI(api_key=api_key)

# Função para extrair texto de um arquivo PDF
def extrair_texto_pdf(caminho_arquivo):
    texto = ""
    try:
        with open(caminho_arquivo, 'rb') as arquivo:
            leitor = PyPDF2.PdfReader(arquivo)
            num_paginas = len(leitor.pages)

            # Limitar a quantidade de páginas para não sobrecarregar o modelo
            paginas_limite = min(num_paginas, 30)

            for i in range(paginas_limite):
                pagina = leitor.pages[i]
                texto += pagina.extract_text() + "\n\n"

        return texto
    except Exception as e:
        print(f"Erro ao ler o PDF {caminho_arquivo}: {str(e)}")
        return ""

# Função para dividir o texto em chunks menores se necessário
def dividir_texto(texto, tamanho_maximo=12000):
    if len(texto) <= tamanho_maximo:
        return [texto]

    # Dividir o texto em parágrafos
    paragrafos = texto.split('\n\n')
    chunks = []
    chunk_atual = ""

    for paragrafo in paragrafos:
        # Se adicionar este parágrafo exceder o tamanho máximo, começar novo chunk
        if len(chunk_atual) + len(paragrafo) > tamanho_maximo and chunk_atual:
            chunks.append(chunk_atual)
            chunk_atual = paragrafo
        else:
            chunk_atual += "\n\n" + paragrafo if chunk_atual else paragrafo

    # Adicionar o último chunk
    if chunk_atual:
        chunks.append(chunk_atual)

    return chunks

# Estrutura para as informações que queremos extrair
campos_extracao = """
Informações Básicas:
- Número do Pregão/Licitação
- Modalidade
- Tipo de Sistema
- Número do Processo Administrativo
- Data da Sessão
- Horário da Sessão
- Local/Portal para realização
- Critério de Julgamento

Informações do Órgão:
- Nome do Órgão
- Seção Responsável
- Endereço do Órgão
- CEP
- Cidade/Estado

Objeto da Licitação:
- Descrição do Objeto
- Divisão em Grupos/Itens
- Regime de Execução

Prazos e Datas:
- Data de Publicação do Edital
- Prazo para Impugnação
- Prazo para Pedidos de Esclarecimento
- Vigência do Contrato/Ata
- Data de Validade das Propostas

Requisitos de Habilitação:
- Habilitações Exigidas
- Qualificação Técnica Específica
- Índices Econômicos Exigidos
- Percentual de Capital Mínimo/Patrimônio Líquido

Garantia e Pagamento:
- Exigência de Garantia
- Condições de Pagamento

Documentação Anexa:
- Lista de Anexos do Edital

Autoridades Responsáveis:
- Nome do Pregoeiro/Presidente da Comissão
- Nome da Autoridade que Aprovou o Edital
- Cargos dos Responsáveis

Legislação Aplicável:
- Leis e Decretos Citados

Restrições e Impedimentos:
- Restrições à Participação
- Vedações Específicas

Informações sobre ME/EPP:
- Tratamento Diferenciado para ME/EPP
- Critérios de Desempate

Informações Financeiras e de Preço:
- Valor Total Estimado da Contratação
- Preço Máximo Aceitável
- Fonte de Recursos/Dotação Orçamentária
- Código UASG do Órgão

Detalhes do Processo de Seleção:
- Modo de Disputa
- Intervalo Mínimo entre Lances
- Critérios de Aceitabilidade de Preços
- Possibilidade de Adesão à Ata por Órgãos Não Participantes
- Quantidade Máxima para Adesão por Órgãos Não Participantes

Requisitos Técnicos e Operacionais:
- Necessidade de Vistoria
- Prazos de Execução/Entrega do Objeto
- Local de Execução/Entrega
- Critérios de Sustentabilidade Ambiental
- Exigência de Amostras/Protótipos

Detalhes sobre a Execução Contratual:
- Possibilidade de Subcontratação
- Forma de Gestão e Fiscalização do Contrato
- Instrumentos de Medição de Resultado (IMR)
- Sanções e Penalidades Específicas
- Condições para Reajuste/Repactuação/Revisão

Informações Documentais:
- Formato dos Documentos a Serem Apresentados
- Exigência de Cadastro Prévio no SICAF
- Prazo para Envio da Documentação Complementar
- Formato das Propostas

Informações Adicionais do Edital:
- Número/Identificação do Edital
- Possibilidade de Consórcio de Empresas
- Possibilidade de Participação de Empresas Estrangeiras
- Exigência de Capital Social Mínimo
- Obrigações Específicas da Contratada e Contratante
- Matriz de Riscos

Informações sobre a Natureza do Objeto:
- Classificação do Serviço
- Categoria do Objeto
- Necessidade de Dedicação Exclusiva de Mão de Obra

Detalhes Temporais:
- Prazo de Garantia do Produto/Serviço
- Cronograma de Execução
- Prazo para Assinatura do Contrato/Ata

Especificações Técnicas:
- Detalhamento das Especificações Técnicas dos Itens/Serviços
- Unidades de Medida
- Quantidades por Item
"""

# Função para extrair informações usando a API da OpenAI
def extrair_informacoes_com_gpt(texto, nome_arquivo):
    prompt = f"""
Analise cuidadosamente o texto do edital de licitação abaixo e extraia as seguintes informações.
Responda SOMENTE em formato JSON, com os campos exatamente como listados abaixo.
Caso não encontre a informação, preencha com "Não informado" ou "N/A".

Campos a serem extraídos:
{campos_extracao}

---
TEXTO DO EDITAL:
{texto}
---

RESPOSTA EM FORMATO JSON:
"""

    try:
        resposta = cliente.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            response_format={"type": "json_object"}
        )

        # Tentar extrair o JSON da resposta
        texto_resposta = resposta.choices[0].message.content.strip()

        try:
            dados_json = json.loads(texto_resposta)
            # Adicionar o nome do arquivo nos dados
            dados_json["Nome do Arquivo"] = nome_arquivo
            return dados_json
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar JSON: {str(e)}")
            print(f"Texto recebido: {texto_resposta[:200]}...")
            return {"Erro": "Falha ao extrair informações", "Nome do Arquivo": nome_arquivo}

    except Exception as e:
        print(f"Erro na API da OpenAI: {str(e)}")
        return {"Erro": str(e), "Nome do Arquivo": nome_arquivo}

# Função para processar vários PDFs
def processar_pdfs_em_massa(diretorio):
    resultados = []
    arquivos_pdf = [arquivo for arquivo in os.listdir(diretorio) if arquivo.lower().endswith('.pdf')]

    print(f"Encontrados {len(arquivos_pdf)} arquivos PDF no diretório.")

    for arquivo in tqdm(arquivos_pdf, desc="Processando PDFs"):
        caminho_completo = os.path.join(diretorio, arquivo)
        texto = extrair_texto_pdf(caminho_completo)

        if not texto:
            print(f"Não foi possível extrair texto de {arquivo}")
            continue

        # Dividir texto em chunks se necessário
        chunks = dividir_texto(texto)

        if len(chunks) > 1:
            print(f"Arquivo {arquivo} dividido em {len(chunks)} partes devido ao tamanho.")

            # Analisar cada chunk e mesclar os resultados
            resultado_combinado = {"Nome do Arquivo": arquivo}

            for i, chunk in enumerate(chunks):
                print(f"Processando parte {i+1} de {len(chunks)} do arquivo {arquivo}")
                resultado_parcial = extrair_informacoes_com_gpt(chunk, f"{arquivo} (parte {i+1})")

                # Mesclar resultados parciais no resultado combinado (priorizando valores não vazios)
                for chave, valor in resultado_parcial.items():
                    if chave != "Nome do Arquivo" and valor not in ["Não informado", "N/A", ""]:
                        resultado_combinado[chave] = valor

                # Pausa para evitar limitação de rate da API
                time.sleep(1)

            resultados.append(resultado_combinado)
        else:
            # Arquivo pequeno o suficiente para ser processado de uma vez
            resultado = extrair_informacoes_com_gpt(texto, arquivo)
            resultados.append(resultado)

            # Pausa para evitar limitação de rate da API
            time.sleep(1)

    return resultados

# Função para processar um único PDF (upload)
def processar_pdf_upload():
    print("Faça o upload do arquivo PDF:")
    uploaded = files.upload()

    if not uploaded:
        print("Nenhum arquivo foi enviado.")
        return None

    nome_arquivo = list(uploaded.keys())[0]

    if not nome_arquivo.lower().endswith('.pdf'):
        print("O arquivo enviado não é um PDF.")
        return None

    texto = extrair_texto_pdf(nome_arquivo)

    if not texto:
        print(f"Não foi possível extrair texto de {nome_arquivo}")
        return None

    chunks = dividir_texto(texto)

    if len(chunks) > 1:
        print(f"Arquivo dividido em {len(chunks)} partes devido ao tamanho.")

        # Analisar cada chunk e mesclar os resultados
        resultado_combinado = {"Nome do Arquivo": nome_arquivo}

        for i, chunk in enumerate(chunks):
            print(f"Processando parte {i+1} de {len(chunks)}")
            resultado_parcial = extrair_informacoes_com_gpt(chunk, f"{nome_arquivo} (parte {i+1})")

            # Mesclar resultados parciais no resultado combinado (priorizando valores não vazios)
            for chave, valor in resultado_parcial.items():
                if chave != "Nome do Arquivo" and valor not in ["Não informado", "N/A", ""]:
                    resultado_combinado[chave] = valor

            # Pausa para evitar limitação de rate da API
            time.sleep(1)

        return resultado_combinado
    else:
        # Arquivo pequeno o suficiente para ser processado de uma vez
        return extrair_informacoes_com_gpt(texto, nome_arquivo)

# Função para salvar resultados em CSV
def salvar_resultados_csv(resultados, nome_arquivo="resultados_licitacoes.csv"):
    if not resultados:
        print("Nenhum resultado para salvar.")
        return

    # Criar um DataFrame pandas com os resultados
    df = pd.DataFrame(resultados)

    # Salvar o DataFrame como CSV
    df.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')
    print(f"Resultados salvos em {nome_arquivo}")

    # Se estiver no Google Colab, disponibilizar o arquivo para download
    try:
        from google.colab import files
        files.download(nome_arquivo)
        print("Arquivo disponível para download.")
    except:
        pass

# Função principal que integra todas as funcionalidades
def main():
    print("=== EXTRATOR DE INFORMAÇÕES DE EDITAIS DE LICITAÇÃO ===")
    print("Este programa extrai informações estruturadas de editais de licitação em PDF.")

    # Testa se a API Key está configurada
    if not api_key:
        print("ERRO: A chave de API da OpenAI não está configurada.")
        print("Adicione-a nas secrets do Google Colab com o nome 'OPENAI_API_KEY'.")
        return

    print("\nEscolha uma opção:")
    print("1. Processar um único arquivo PDF (upload)")
    print("2. Processar múltiplos arquivos PDF de um diretório do Google Drive")

    opcao = input("\nOpção (1 ou 2): ")

    if opcao == "1":
        # Processar um único arquivo
        resultado = processar_pdf_upload()
        if resultado:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nome_arquivo = f"resultado_licitacao_{timestamp}.csv"
            salvar_resultados_csv([resultado], nome_arquivo)

    elif opcao == "2":
        # Processar múltiplos arquivos
        print("\nDigite o caminho do diretório no Google Drive (ex: /content/drive/MyDrive/licitacoes):")
        diretorio = input("Caminho: ")

        if not os.path.exists(diretorio):
            print(f"O diretório {diretorio} não existe.")
            return

        resultados = processar_pdfs_em_massa(diretorio)

        if resultados:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nome_arquivo = f"resultados_licitacoes_{timestamp}.csv"
            salvar_resultados_csv(resultados, nome_arquivo)

    else:
        print("Opção inválida.")

# Executar o programa
if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== EXTRATOR DE INFORMAÇÕES DE EDITAIS DE LICITAÇÃO ===
Este programa extrai informações estruturadas de editais de licitação em PDF.

Escolha uma opção:
1. Processar um único arquivo PDF (upload)
2. Processar múltiplos arquivos PDF de um diretório do Google Drive

Opção (1 ou 2): 2

Digite o caminho do diretório no Google Drive (ex: /content/drive/MyDrive/licitacoes):
Caminho: /content/drive/MyDrive/_Blog & Newsletter/Cursos/Curso IA Aplicada/licitacoes
Encontrados 11 arquivos PDF no diretório.


Processando PDFs:   0%|          | 0/11 [00:00<?, ?it/s]

Arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf dividido em 7 partes devido ao tamanho.
Processando parte 1 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Processando parte 2 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Processando parte 3 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Processando parte 4 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Processando parte 5 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Processando parte 6 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Processando parte 7 de 7 do arquivo Edital - Pregao Eletronico 036-2022 - Concessao restaurante - COMPRASNET.pdf
Arquivo SEI_TJDFT - 3053799 - Edital.pdf dividido em 12 partes devido ao tamanho.
Pr